In [53]:
import json
from shapely.geometry import shape, GeometryCollection, Point
from typing import Tuple
#Documentation: https://shapely.readthedocs.io/en/stable/

In [54]:
#read geojson file for wojewodztwo level
with open('wojewodztwa-max.geojson.txt', "r", encoding="utf-8") as f:
     wojewodztwa = json.load(f)
#read geojson file for powiat level
with open('powiaty-max.geojson.txt', "r", encoding="utf-8") as f:
     powiaty = json.load(f)

In [59]:
def look_up(longitude: float, latitude:float ) -> Tuple[str,str] :
    
    point: Point = Point(longitude, latitude)
    
    wojewodztwo: str = 'not in PL'
    powiat: str = 'not in PL'
    
    for feature in wojewodztwa['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            wojewodztwo = feature['properties']['nazwa']
    
    for feature in powiaty['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            powiat = feature['properties']['nazwa']
        
    return wojewodztwo, powiat


In [60]:
info: Tuple[str,str] = look_up(20.975166, 50.173166 )

In [57]:
print(info)

('małopolskie', 'powiat dąbrowski')
